In [281]:
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
import requests
import io
import tabula
import pandas as pd
import numpy as np

In [ ]:
BASE_URL = "https://dijlovasok.hu/index.php/programgyujt"

In [ ]:
def get_contents_of_page (url, headers = None) :
    req = requests.get(url, headers = headers if headers != None else {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
    })
    
    return req.content

In [ ]:
def soupify (content) :
    return BeautifulSoup(content, "html.parser")

In [ ]:
def get_links_from_main_page ():
    soup = soupify(get_contents_of_page(BASE_URL))
    # print(soup.prettify())
    program_hrefs = list(
        set(
            map(
                lambda href: href["href"], 
                filter(lambda href: href["href"].startswith("/index.php/programgyujt/"), 
                    soup.find_all("a", href = True)
                )
            )
        )
    )
    return program_hrefs    

In [ ]:
def get_pdf_links_from_page (url):
    soup: BeautifulSoup = soupify(get_contents_of_page(url))
    pdf_hrefs = list(
            set(
                map(
                    lambda href: href["href"], 
                    filter(lambda href: href["href"].endswith(".pdf"), 
                        soup.find_all("a", href = True)
                    )
                )
            )
        )
    return pdf_hrefs

In [220]:
def get_pdf_dataframes () :

    pdf_texts = {}

    for page_url in get_links_from_main_page() :
        
        print(f"Scraping page {page_url}...")
        
        page_content = {}
        
        for pdf_url in get_pdf_links_from_page(BASE_URL + page_url) :
            
            print(f"Downloading pdf {pdf_url}...")
                        
            user_agent= 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'

            dfs = tabula.read_pdf(
                pdf_url, 
                multiple_tables=True, 
                pages="all", 
                guess=True,
                encoding="cp1252",
                user_agent=user_agent,
                # output_format="dataframe",
                pandas_options={
                    # "names": ["ID", "Position", "Description", "Notes"]
                    "header": 0
                },
                lattice=True,
                silent=True
            )   
                
            if len(dfs) == 0 :
                continue
                
            page_content[pdf_url] = dfs
            
        pdf_texts[page_url] = page_content
        
    return pdf_texts

In [221]:
database = get_pdf_dataframes()

Scraping page /index.php/programgyujt/1457-kozep-osztalyu-feladatok-mb-program...
Scraping page /index.php/programgyujt/1031-elokeszito-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1217-vezetoszaras-feladat...
Scraping page /index.php/programgyujt/1035-kozep-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1008-fiatal-lovak-programjai...
Scraping page /index.php/programgyujt/1034-konnyu-kozep-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1037-nagydij-szintu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1033-konnyu-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1032-alapfoku-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1036-nehez-osztalyu-feladatok-hatalyos-2018-05-01...


In [282]:
def inspect_database(database) :
    for page_url in database:
        page = database[page_url]
        for program_url in page:
            program = page[program_url]
            for df in program:
                yield df

In [283]:
next(inspect_database(database))

,Unnamed: 0


In [259]:
list(list(database.values())[0].values())[3][1]

,Unnamed: 0,Unnamed: 1,Feladat,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Megjegyzés
0,1.0,A\r\rX\r\rC,"Belovaglás összeszedett\rügetésben\rÁllj, kösz...",NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,H-E,Vállat be,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,E-X\rX-B,"Fél kiskör balra, 10m\rFél kiskör jobbra, 10m",NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,B-F,Vállat be,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,A\rL,"Középbõl\rKiskör jobbra, 10m",NaN,NaN,NaN,NaN,NaN,NaN
5,6.0,L-R,Jobbra oldaljárás,NaN,NaN,NaN,2.0,NaN,NaN
6,7.0,C,"Állj, 5 lépést hátra lépj, abból\rösszeszedett...",NaN,NaN,NaN,NaN,NaN,NaN
7,8.0,H-K\rK,Középügetés\rÖsszeszedett ügetés,NaN,NaN,NaN,NaN,NaN,NaN
8,9.0,A\rL,"Középbõl\rKiskör balra, 10m",NaN,NaN,NaN,NaN,NaN,NaN
9,10.0,L-S,Balra oldaljárás,NaN,NaN,NaN,2.0,NaN,NaN


In [460]:
from pandas import DataFrame
from typing import List

def convert_program_dfs_to_useful_df (dfs: List[DataFrame]) :
            
    dfs = list(filter(lambda df: len(df) != 0, dfs))
    
    if len(dfs) == 0 :
        return None
    
    if len(dfs) == 1 :
        if len(dfs[0]) != 0 :
            return dfs[0]
        else :
            return None
    
    # joining
    
    result = dfs[0]
    
    column_num = len(result.columns)
    
    for df in dfs[1:] :
        
        if len(df.columns) == column_num :
            result = pd.concat([result, df], ignore_index = True)
            continue
        
        if len(df.columns) == column_num + 2 :
            
            columns = df.columns
            
            df = df.drop(columns[0], axis=1)
            df = df.drop(columns[-1], axis=1)
            
            # rename unnamed columns
            
            def column_renamer(column):
                if str(column).startswith("Unnamed") == False :
                    return column
                num = int(column.split(" ")[1]) - 1
                return "Unnamed: " + str(num)
            
            df = df.rename(column_renamer, axis=1)
            
            result = pd.concat([result, df], ignore_index = True)
            continue
        
        # raise Exception(f"WARNING: df column width mismatch: cannot append {len(df.columns)} to {column_num}!")
        
    return result

In [461]:

def convert_database_to_useful_dfs (database) :
    for page in database.values() :
        for program_url in page :
            program = page[program_url]
            try :
                yield convert_program_dfs_to_useful_df(program)
            except Exception as e :
                print(f"URL in question: {program_url}")
                raise e

In [578]:
def is_df_program(df) :
    if "Feladat" not in df.columns :
        return False
    
    if len(df) <= 5 :
        return False
    
    if "Négyszöget" not in df["Feladat"][pd.Series.last_valid_index(df["Feladat"])] :
        return False
    
    return True

In [579]:
iter = filter(is_df_program, convert_database_to_useful_dfs(database))

In [633]:
next(iter)

StopIteration: 